In [2]:
# Imports
import pandas as pd
import numpy as np
#from scipy.stats import entropy

# Question 1
load data is_sleepy.csv to a dataframe

(a) what is the entropy of 'meal' 

In [96]:
df_is_sleepy = pd.read_csv('is_sleepy.csv')
df_is_sleepy = df_is_sleepy.set_index(df_is_sleepy.columns[0])
df_is_sleepy.head()

,meal,tired,exercise,sleepy
Unnamed: 0,,,,
0,0,1,1,0
1,1,1,0,0
2,0,0,1,0
3,0,1,1,0
4,0,1,1,0


In [97]:

# returns the entropy of the parameter column in df
def calc_entropy(df, col_name):
    col_values = df[col_name].ravel()
    count_values = col_values.shape[0]
    entropy = 0
    for val in np.unique(col_values):
        proba_val = col_values[np.where(col_values == val)].shape[0] / count_values
        entropy += proba_val * np.log2(proba_val)
    
    return -entropy

print("The entropy of 'meal' is %.4f" % calc_entropy(df_is_sleepy, 'meal'))


The entropy of 'meal' is 0.9427


(b) what is the entropy of 'tired'

In [98]:
print("The entropy of 'tired' is %.4f" % calc_entropy(df_is_sleepy, 'tired'))


The entropy of 'tired' is 0.9044


(c) what is the conditional entropy of 'meal' given 'tired'

In [136]:
# returns the conditional entropy H(Y | X)
def conditional_entropy(df, X, Y):
    cond_entropy = 0
    for x in np.unique(df[X]):
        df_x = df.loc[df[X] == x]
        for y in np.unique(df[Y]):
            df_x_and_y = df_x.loc[df_x[Y] == y]
            P_x_and_y = df_x_and_y.shape[0] / df.shape[0]
            P_y_given_x = df_x_and_y.shape[0] / df_x.shape[0]
                        
            cond_entropy += P_x_and_y * np.log2(P_y_given_x)
            
    return - cond_entropy

print("The entropy of 'meal' given 'tired' is %.4f" % conditional_entropy(df_is_sleepy, 'tired', 'meal'))


The entropy of 'meal' given 'tired' is 0.8407


(d) look at the definition of mutual information from the lecture and answer

$$ I(X;Y) \overset{?}{=} I(Y;X) $$

<center>$I(X;Y) = \underset{x}{\sum}\underset{y}{\sum}P(X,Y)log\dfrac{P(X,Y)}{P(X)P(Y)}$</center>
By symmetry, the joint probability $P(X, Y) = P(Y, X)$

It follows that
<center>$\boxed{I(X;Y) = I(Y;X)}$</center>

(e) compute 
$$ I(X;Y)=H(X)-H(X\mid Y) $$
and 
$$ I(Y;X)=H(Y)-H(Y\mid X) $$
for X=meal and Y=tired
are they equal? do they satisfy 
$$ \left\|I(Y;X)-I(X;Y)\right\|<1\cdot 10^{-15} $$

In [137]:
# returns I(X; Y)
def mutual_information(df, X, Y):
    return calc_entropy(df, X) - conditional_entropy(df, Y, X)

mut_inf_X_Y = mutual_information(df_is_sleepy, 'meal', 'tired')
mut_inf_Y_X = mutual_information(df_is_sleepy, 'tired', 'meal')

print('I(X; Y) = %.4f, I(Y; X) = %.4f, |I(Y; X) - I(X; Y)| < 1e-15 = %r' % (mut_inf_X_Y, mut_inf_Y_X, abs(mut_inf_X_Y-mut_inf_Y_X) < 1e-15))


I(X; Y) = 0.1020, I(Y; X) = 0.1020, |I(Y; X) - I(X; Y)| < 1e-15 = True


# Question 2
use the same data from question 1. our target is to predict sleepy column using the other columns

(a) what is the optimal question for the data $ qa $ using the information gain criterion?

In [138]:
def best_question(df, target_col):
    arg_max = 0
    qa = ''
    for col_name in df.drop(target_col, axis=1).columns:
        current_I = mutual_information(df, target_col, col_name)
        if arg_max < current_I:
            arg_max = current_I
            qa = col_name
    return qa, arg_max

qa, mut_info = best_question(df_is_sleepy, 'sleepy')
print("The optimal qa using Information Gain criterion is '%s' with I('sleepy','%s') = %.4f" % (qa, qa, mut_info))
    

The optimal qa using Information Gain criterion is 'tired' with I('sleepy','tired') = 0.1550


(b) we segment the data as following

\begin{array}{|c|c|c|}
\hline
\boldsymbol{math} & \boldsymbol{python} & \textbf{in words} \\
\hline
\hline
\hline
D_{1y} & D.qa==1 & qa\text{ is yes}\\
\hline
D_{1n} & D.qa==0 & qa\text{ is no}\\
\hline
\end{array}


you can now ask another question $ qb $
you can ask it regarding $ D_{1y} $ or $ D_{1n} $

What is the best question for $ D_{1y} $? What is the best question for $ D_{1n} $?

In [141]:
df_D1y = df_is_sleepy.loc[df_is_sleepy[qa] == 1]
df_D1n = df_is_sleepy.loc[df_is_sleepy[qa] == 0]

qb_D1y, mut_info_D1y = best_question(df_D1y, 'sleepy')
print("The optimal qb when qa is 'yes' using Information Gain criterion is '%s' with I('sleepy','%s') = %.4f" % (qb_D1y, qb_D1y, mut_info_D1y))
qb_D1n, mut_info_D1n = best_question(df_D1n, 'sleepy')
print("The optimal qb when qa is 'no' using Information Gain criterion is '%s' with I('sleepy','%s') = %.4f" % (qb_D1n, qb_D1n, mut_info_D1n))

The optimal qb when qa is 'yes' using Information Gain criterion is 'exercise' with I('sleepy','exercise') = 0.0540
The optimal qb when qa is 'no' using Information Gain criterion is 'exercise' with I('sleepy','exercise') = 0.1058


**Bonus**

(c) if you can only ask the question $ q2 $ for either $ D_{1y} $ or $ D_{1n} $, which one should you choose?

I would chose qb when qa is 'no' because it has a better information gain